In [14]:
import numpy as np
import cv2

vc = cv2.VideoCapture('SmallVideo.mp4')
_, first_frame = vc.read()
# Scale and resize image
resize_dim = 600
max_dim = max(first_frame.shape)
scale = resize_dim/max_dim
first_frame = cv2.resize(first_frame, None, fx=scale, fy=scale)
# Convert to gray scale 
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

print(first_frame.shape)
# Create mask
mask = np.zeros_like(first_frame)
# Set image saturation to maximum value as we do not need it
mask[:, 1] = 255

while(vc.isOpened()):
 # Read a frame from video
    _, frame = vc.read()
 
 # Convert new frame format`s to gray scale and resize gray frame obtained
    frame = cv2.resize(frame, None, fx=scale, fy=scale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# Calculate dense optical flow by Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, pyr_scale = 0.5, levels = 5, winsize = 11, iterations = 5, poly_n = 5, poly_sigma = 1.1, flags = 0)
 # Compute the magnitude and angle of the 2D vectors
    magnitude, angle = cv2.cartToPolar(flow[:,:, 0], flow[:,:, 1])
 # Set image hue according to the optical flow direction
    mask[:,:, 0] = angle * 180 / np.pi / 2
 # Set image value according to the optical flow magnitude (normalized)
    mask[:,:, 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
 # Convert HSV to RGB (BGR) color representation
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    
# Open a new window and displays the output frame
    dense_flow = cv2.addWeighted(frame, 1,rgb, 2, 0)
    cv2.imshow("Dense optical flow", dense_flow)
 # Update previous frame
    prev_gray = gray
 # Frame are read by intervals of 10 millisecond. The programs breaks out of the while loop when the user presses the ‘q’ key
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

(316, 600, 3)


error: OpenCV(4.1.1) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
